# E-commerce Sales Insights

This notebook demonstrates SQL-based data analysis using a synthetic e-commerce dataset loaded into a SQLite database.

In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Connect to the database
conn = sqlite3.connect("ecommerce.db")


ModuleNotFoundError: No module named 'pandas'

## Top 10 Customers by Total Spending

In [ ]:
query = '''
SELECT c.customer_id, c.name, SUM(o.total_amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 10;
'''
top_customers = pd.read_sql_query(query, conn)
top_customers

## Monthly Revenue Trend

In [ ]:
query = '''
SELECT substr(order_date, 1, 7) AS month, SUM(total_amount) AS revenue
FROM orders
WHERE status IN ('Shipped', 'Delivered')
GROUP BY month
ORDER BY month;
'''
monthly_revenue = pd.read_sql_query(query, conn)
monthly_revenue.plot(x='month', y='revenue', kind='line', title='Monthly Revenue', figsize=(10, 5))
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


## Most Returned Products

In [ ]:
query = '''
SELECT p.product_name, COUNT(*) AS return_count
FROM returns r
JOIN products p ON r.product_id = p.product_id
GROUP BY p.product_name
ORDER BY return_count DESC
LIMIT 10;
'''
returns = pd.read_sql_query(query, conn)
returns

## Repeat Customer Percentage

In [ ]:
query = '''
WITH order_counts AS (
  SELECT customer_id, COUNT(*) AS order_count
  FROM orders
  GROUP BY customer_id
)
SELECT
  ROUND(100.0 * SUM(CASE WHEN order_count > 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS repeat_customer_percentage
FROM order_counts;
'''
repeat_rate = pd.read_sql_query(query, conn)
repeat_rate

## Profit by Product

In [ ]:
query = '''
SELECT 
  p.product_name, 
  SUM(oi.quantity * (oi.item_price - p.cost)) AS total_profit
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_profit DESC
LIMIT 10;
'''
profit = pd.read_sql_query(query, conn)
profit

In [ ]:
conn.close()